In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!wget https://github.com/camilocarvajalreyes/entidades-minsal/raw/main/datos/Etiquetado/corpus_100k_test.txt
!wget https://github.com/camilocarvajalreyes/entidades-minsal/raw/main/datos/Etiquetado/corpus_100k_train.txt

--2022-11-21 07:45:42--  https://github.com/camilocarvajalreyes/entidades-minsal/raw/main/datos/Etiquetado/corpus_100k_test.txt
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/camilocarvajalreyes/entidades-minsal/main/datos/Etiquetado/corpus_100k_test.txt [following]
--2022-11-21 07:45:43--  https://raw.githubusercontent.com/camilocarvajalreyes/entidades-minsal/main/datos/Etiquetado/corpus_100k_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2040487 (1.9M) [text/plain]
Saving to: ‘corpus_100k_test.txt.1’

corpus_100k_test.tx 100%[===================>]   1.95M  --.-KB/s    in 0.07s   


In [ ]:
import numpy as np

a = np.loadtxt('corpus_100k_train.txt', dtype= object,delimiter= '_[X]_' )
b = np.loadtxt('corpus_100k_test.txt', dtype= object,delimiter= '_[X]_' )

In [ ]:
#Descarga de datos
!gdown --id '1vcYj7xDwH38AZ8L2AHp-zdqddK-KeaVI'
!gdown --id '1pyy6OWkNxbLsfaIRqzMD06aGqPTIc_DG'

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1vcYj7xDwH38AZ8L2AHp-zdqddK-KeaVI
To: /content/DATA_HLF_MDS_2.csv
100% 477M/477M [00:03<00:00, 150MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1pyy6OWkNxbLsfaIRqzMD06aGqPTIc_DG
To: /content/PRINCIPIOS_ACTIVOS_MDS.xlsx
100% 69.2k/69.2k [00:00<00:00, 19.2MB/s]


In [ ]:
#Al importar los datos estan corridos, se puede ver en Resumen y FormaFrama, donde estan corridos en una posicion hacia abajo
main_db = pd.read_csv('/content/DATA_HLF_MDS_2.csv',sep=',')

In [ ]:
code_db = pd.read_excel('/content/PRINCIPIOS_ACTIVOS_MDS.xlsx')
HLF = code_db.loc[:,['PRINCIPIO_ACTIVO','CODIGO_HLF']]

In [ ]:
HLF2 = HLF['CODIGO_HLF'].str.split(', ')
HLF_list = []
for i in range(len(HLF2)):
  HLF_list = HLF_list + HLF2[i] 
try:
  HLF_list.remove('')
except:
  pass
HLF_name = []
for elem in HLF_list:
  HLF_name = HLF_name + HLF[HLF['CODIGO_HLF'].str.contains(elem)]['PRINCIPIO_ACTIVO'].values.tolist()

In [ ]:
HLF_df = pd.DataFrame()
HLF_df['CODIGO_MEDICAMENTO'] = HLF_list
HLF_df['PRINCIPIO_ACTIVO'] = HLF_name

In [ ]:
df = main_db.join(HLF_df.set_index('CODIGO_MEDICAMENTO'), on='CODIGO_MEDICAMENTO')

In [ ]:
PA = sorted(df['PRINCIPIO_ACTIVO'].dropna().unique(), key=len, reverse=True)
FF = sorted(df['FORMA_FARMA'].dropna().unique(), key=len, reverse=True)
ADMIN = 'ORAL|INTRAVENOSA|INTRAVASCULAR|INTRAMUSCULAR|INTRATECAL|SUCUT.NEA|SUBLINGUAL|BUCAL|RECTAL|VAGINAL|OCULAR|.TICA|NASAL|INHALACI.N|NEBULIZACI.N|CUT.NEO|T.PIC.|TRANSD.RMICO'

In [ ]:
def Reg_expresion(txt,PA,FF,ADMIN):
  for elem in PA:
    if re.search(elem.upper(), txt.upper()):
      txt = re.sub(elem.upper(), '[B_ACTIVE_PRINCIPLE]'+elem+'[E_ACTIVE_PRINCIPLE]', txt)
      break
  for elem in FF:
    if re.search(elem.upper(), txt.upper()):
      txt =re.sub(elem.upper(), '[B_FORMA_FARMA]'+elem+'[E_FORMA_FARMA]',txt,1)
      break
  if re.search(r'cada [0-9].? d.as|cada [0-9].? horas|cada [0-9].? minutos|al acostarse|diari.|(\w+) veces al d.a',txt):
    elem = re.search(r'cada [0-9].? d.as|cada [0-9].? horas|cada [0-9].? minutos|al acostarse|diari.|(\w+) veces al d.a',txt).group()
    txt = re.sub(elem, '[B_PERIODICITY]'+elem+'[E_PERIODICITY]',txt,1)

  if re.search('durante .+ d.as',txt):
    elem = re.search('durante .+ d.as',txt).group()
    txt = re.sub(elem, '[B_DURATION]'+elem+'[E_DURATION]',txt,1)

  if re.search(ADMIN,txt):
    elem = re.search(ADMIN,txt).group()
    txt = re.sub(elem, '[B_ADMIN]'+elem+'[E_ADMIN]',txt,1)
  return txt

In [ ]:
def generate_column_o(corpus):
  n = 0
  for i in range(len(corpus)):
    c = corpus[i].split()
    for elem in c:
      n+=1
    n+=1
  output = np.empty([n,2], dtype= object)
  idx = 0
  for i in range(len(corpus)):
    c= corpus[i].split()
    j=0
    while j < len(c):
      output[idx] = np.array([c[j], 'O'],dtype = object)
      idx += 1
      j+=1
    output[idx] = np.array(['', ''],dtype = object)
    idx += 1
  return output

In [ ]:
def etiquetar(corpus, PA,FF,ADMIN):
  output = np.empty([len(corpus)],dtype = object)
  for i in range(len(corpus)):
    try:
      output[i] = Reg_expresion(corpus[i],PA,FF,ADMIN)
    except:
      output[i] = corpus[i]
  return output

In [ ]:
def put_ticket(corpus):
  print = 'O'
  for i in range(len(corpus)):
    if '[B_ACTIVE_PRINCIPLE]' in corpus[i,0] and '[E_ACTIVE_PRINCIPLE]' in corpus[i,0]:
      corpus[i,1] = 'B-ACTIVE_PRINCIPLE'
      corpus[i,0] = corpus[i,0].split(']')[1].split('[')[0]
      print = 'O'
    elif '[B_ACTIVE_PRINCIPLE]' in corpus[i,0]:
      corpus[i,1] = 'B-ACTIVE_PRINCIPLE'
      corpus[i,0] = corpus[i,0].split(']')[1]
      print = 'I-ACTIVE_PRINCIPLE'
    elif '[E_ACTIVE_PRINCIPLE]' in corpus[i,0]:
      corpus[i,1] = 'I-ACTIVE_PRINCIPLE'
      corpus[i,0] = corpus[i,0].split('[')[0]
      print = 'O'
    elif '[B_FORMA_FARMA]' in corpus[i,0] and '[E_FORMA_FARMA]' in corpus[i,0]:
      corpus[i,1] = 'B-FORMA_FARMA'
      corpus[i,0] = corpus[i,0].split(']')[1].split('[')[0]
      print = 'O'
    elif '[B_FORMA_FARMA]' in corpus[i,0]:
      corpus[i,1] = 'B-FORMA_FARMA'
      corpus[i,0] = corpus[i,0].split(']')[1]
      print = 'I-FORMA_FARMA'
    elif '[E_FORMA_FARMA]' in corpus[i,0]:
      corpus[i,1] = 'I-FORMA_FARMA'
      corpus[i,0] = corpus[i,0].split('[')[0]
      print = 'O'
    elif '[B_ADMIN]' in corpus[i,0] and '[E_ADMIN]' in corpus[i,0]:
      corpus[i,1] = 'B-ADMIN'
      corpus[i,0] = corpus[i,0].split(']')[1].split('[')[0]
      print = 'O'
    elif '[B_ADMIN]' in corpus[i,0]:
      corpus[i,1] = 'B-ADMIN'
      corpus[i,0] = corpus[i,0].split(']')[1]
      print = 'I-ADMIN'
    elif '[E_ADMIN]' in corpus[i,0]:
      corpus[i,1] = 'I-ADMIN'
      corpus[i,0] = corpus[i,0].split('[')[0]
      print = 'O'
    elif '[B_PERIODICITY]' in corpus[i,0] and '[E_PERIODICITY]' in corpus[i,0]:
      corpus[i,1] = 'B-PERIODICITY'
      corpus[i,0] = corpus[i,0].split(']')[1].split('[')[0]
      print = 'O'
    elif '[B_PERIODICITY]' in corpus[i,0]:
      corpus[i,1] = 'B-PERIODICITY'
      corpus[i,0] = corpus[i,0].split(']')[1]
      print = 'I-PERIODICITY'
    elif '[E_PERIODICITY]' in corpus[i,0]:
      corpus[i,1] = 'I-PERIODICITY'
      corpus[i,0] = corpus[i,0].split('[')[0]
      print = 'O'
    elif '[B_DURATION]' in corpus[i,0] and '[E_DURATION]' in corpus[i,0]:
      corpus[i,1] = 'B-DURATION'
      corpus[i,0] = corpus[i,0].split(']')[1].split('[')[0]
      print = 'O'
    elif '[B_DURATION]' in corpus[i,0]:
      corpus[i,1] = 'B-DURATION'
      corpus[i,0] = corpus[i,0].split(']')[1]
      print = 'I-DURATION'
    elif '[E_DURATION]' in corpus[i,0]:
      corpus[i,1] = 'I-DURATION'
      corpus[i,0] = corpus[i,0].split('[')[0]
      print = 'O'
    elif corpus[i,0] == '':
      corpus[i,1] = ''
    else:
      corpus[i,1] = print

In [ ]:
corpus_ER_train = generate_column_o(etiquetar(a,PA,FF,ADMIN))
put_ticket(corpus_ER_train)
corpus_ER_test = generate_column_o(etiquetar(b,PA,FF,ADMIN))
put_ticket(corpus_ER_test)

In [ ]:
np.savetxt('corpus_ER_train.txt',corpus_ER_train,fmt="%s")
np.savetxt('corpus_ER_test.txt',corpus_ER_test,fmt="%s")